In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
import numpy as np
from nltk.corpus import words
nltk.download('words')
from nltk.corpus import brown
nltk.download('brown')
from numpy import genfromtxt
from tensorflow.keras import regularizers
import time
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pickle
import random

2021-12-17 03:51:12.937958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-17 03:51:12.937999: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
with open("file_lst.pkl", "rb") as f:
    lst_from_pkl = pickle.load(f)
random.shuffle(lst_from_pkl)
sentence_label_array = []

for name in lst_from_pkl:
    idx = name.index("_")
    label = int(name[:idx]) + 1
    with open(name, 'r') as file:
        data = [line.strip() for line in file.readlines()]
    labels = [label] * len(data)
    sentence_label_array.extend([(data[i],labels[i]) for i in range(len(data))])

random.shuffle(sentence_label_array)
text_train, text_test = train_test_split(sentence_label_array, test_size=0.1)

X_train = [group[0] for group in text_train]
y_train = np.array([group[1] for group in text_train])
X_test = [group[0] for group in text_test]
y_test = np.array([group[1] for group in text_test])

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
with open('tokenizer.pkl','wb') as f:
    pickle.dump(tokenizer,f)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_seq_padded = pad_sequences(X_train_seq, maxlen=500)
X_test_seq_padded  = pad_sequences(X_test_seq, maxlen=500)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(500,)))
model.add(keras.layers.Embedding(input_dim = 10000, output_dim = 450))
model.add(keras.layers.LSTM(150, activation='tanh'))
model.add(keras.layers.Dense(75, activation='relu'))
#model.add(keras.layers.Dense(50, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-3),
    #bias_regularizer=regularizers.l2(1e-3),
    #activity_regularizer=regularizers.l2(1e-5)))
model.add(keras.layers.Dropout(0.05))
model.add(keras.layers.Dense(3, activation='softmax'))
model.summary()
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=train_acc_metric)
model.fit(x=X_train_seq_padded,y=y_train,batch_size=32,epochs=10,verbose=2,validation_data=(X_test_seq_padded,y_test))


2021-12-17 03:51:59.449241: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-17 03:51:59.449304: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-17 03:51:59.449335: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cluster-99b4-m): /proc/driver/nvidia/version does not exist
2021-12-17 03:51:59.449719: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 450)          4500000   
                                                                 
 lstm (LSTM)                 (None, 150)               360600    
                                                                 
 dense (Dense)               (None, 75)                11325     
                                                                 
 dropout (Dropout)           (None, 75)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 228       
                                                                 
Total params: 4,872,153
Trainable params: 4,872,153
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
316/316 - 260s - loss: 0.8701 - spa

In [4]:
model.save(filepath = "./model")

2021-12-17 04:39:18.690583: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: ./model/assets
